# Runke Kutta with Linear Programming

The goal of ths spreadsheed is too test methods to determin the b-Coefficients of the Runge Kutta in every step. The solution of every step should comply with $y \geq 0$
As second condition they should comply with the order conditions

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from nodepy import rk
import cvxpy as cp


import numpy.linalg as linalg


rk4 = rk.loadRKM('RK44').__num__()
rk4x2 = rk4*rk4
ssp2 = rk.loadRKM('SSP22').__num__()
ssp3 = rk.loadRKM('SSP33').__num__()
ssp104 = rk.loadRKM('SSP104').__num__()
merson4 = rk.loadRKM('Merson43').__num__()
bs5 = rk.loadRKM('BS5').__num__()

trbdf = rk.loadRKM('TR-BDF2').__num__()
be = rk.loadRKM('BE').__num__()
irk2 = rk.loadRKM('LobattoIIIA2').__num__()

## Set up Order Conditions

The Order Conditions are an linear equations system and can be writen as $O b = r$. 

The vector $b \in R^s$ contains the b coefficents. $s$ is the number of stages. $O$ is an $n \times s$ matrix, where $g$ is the number of Order Conditions. $r$ is an vector containing the right hand side of the Order Conditions

Has Solution ->
No contradiction -> can be avoided by choosing right $A$ matrix (and $c$) 



In order to get an optimisation problem the equation system has to be underdetermined.

This implies that $rank{O} \leq s$. 

Are the Order Conditions usually linear independant?
If so: $rank{O} = g \leq s$









In [ ]:
def tau(k,A,c): 
    #generates tau vector 
    return 1/ np.math.factorial(k)*c**k - 1/np.math.factorial(k-1) *A @ c**(k-1)
    
def tau_hat(k,A,c):
    return c**k-k*A@(k-1)


def OrderCond(A,c,order = 1):
    #Generates Order Condition Matrix O and right side vector r for Linear Equation System O@b=r
    
    s = len(c) #number of stages
    
    r = []
    O_rows = []
    
    
    if A.shape != (s,s):
        raise InputError
        
    else:
        if order >= 1:
            O_rows.append(np.ones(s));      r.append(1)
            
        if order >= 2:
            O_rows.append(c);               r.append(1/2)
            
        if order >= 3:
            O_rows.append(c**2);            r.append(1/3)
            O_rows.append(tau(2,A,c));      r.append(0.)
            
        if order >= 4:
            O_rows.append(c**3);            r.append(1/4)
            O_rows.append(tau(2,A,c)*c);    r.append(0.)
            O_rows.append(tau(2,A,c)@A.T);  r.append(0.)
            O_rows.append(tau(3,A,c));      r.append(0.)
        
        if order >= 5:
            print('too high order')
            raise NotImplementedError
        
        O = np.vstack(O_rows)
        return (O,np.array(r))
            
                
    
    


In [ ]:
#Test Function
rkm = ssp104
O, rhs = OrderCond(rkm.A,rkm.c,order = rkm.p)

print(O@rkm.b-rhs)


In [ ]:
#Are O vectors linear dependand? -> Rank of O
print ('Size of O:')
print (O.shape)

print('Rank')
print(linalg.matrix_rank(O))

Note: Order is smaler than minimum dimention of Matrix -> some way to simplify the Linear Equation System

## Positifity Constraint  

For solving an PDE with the method of lines an spartial grid  with $p$ points is introduced. 
From this we get an ODE with an $u \in R^p$.

To enshure that the solution is positive 

$ u_i^{n+1} \geq 0   \;   \forall_{i \in \{0,p-1 \}}  $

$ u_i^n + h \sum_{j=0}^{s-1} b_j h_i^j  \geq 0   \;   \forall_{i \in \{0,p-1 \}}  $


has to be fulfilled.
This infers $p$ positivity constraints to the optimisation problem. These can be written as

$u_i + h K \bullet b \geq 0$     (where $K = \big[k^1 , \cdots k^{s-1}\big] $)  



## Objective Function

Different Choices of objective function. Ideas:

$\bullet$ Penalty on large b's 

$\bullet$ Some kind of linear estimation of error term


## Impelmentation of RK method 

Setup: Calculate Order Condition Matrix and right hand side vector
 

a) calculate Intermediate Steps for an Runge Kutta method with arbitrary f(t,u) A and c

b) Pack the calculated Data into a Linear optimisation Problem 

   $\bullet$ Constraints:
   
   Order Condition: $Ob=r$
   
   Positifity: $u_i + h K \bullet b \geq 0$

   $\bullet$ Goal function:
   Different ideas ?
   
    


In [ ]:
def RK_variable_b(rkm, dt, f, w0=[1.,0], t_final=1.,b_fixed = False):
    """    
    Options:
    
        rkm: Base Runge-Kutta method, in Nodepy format
        dt: time step size
        f: RHS of ODE system
        w0: Initial data
        t_final: final solution time        
    """
    
    #setup Variables for Soulution storage
    p = len(w0) #number of dimentions
    
    uu = np.zeros([p,int(t_final/dt)+100])
    uu[:,0] = w0.copy()
    tt = np.zeros([int(t_final/dt)+100])
    
    
    #Setup Runge Kutta 
    c = rkm.c
    A = rkm.A #has to be lower left triangle
    s = len(c) #number of Stages
    K = np.zeros([p,s])
    
    u = np.array(w0)
    t = 0.
    n = 0
    
    
    #Setup Optimisation Problem
    O, rhs = OrderCond(rkm.A,rkm.c,order = rkm.p)
    b_op =cp.Variable(s)
    e = np.ones(s) #vector for gola Fnction, just generates the 1-Norm of b
    
          #Maybee set up Problem here and treat H as an Paramter
        
    #for debugging b's    
    bb = np.zeros([s,int(t_final/dt)+2])
        
    #print('set up starting to solve')
    
    #Solve ODE
    while t<t_final:
        for i in range(s):
            #compute Stages
            
                
            #K[:,i] = f(t+c[i]*dt,u+dt*K@A[i,:]) 
            #the 0s in A should make shure that no data from an older Step is used
            
            #Maybe better Approach, because A[i,j] = 0 in many places
            u_prime = u.copy()
            for m in range(i):
                u_prime += dt*A[i,m]*K[:,m]
            
            K[:,i] = f(t+c[i]*dt,u_prime)
            
            #print('intermediatestep computed')
        
        if b_fixed == False:
            #Run Optimisation Problem
        
            prob = cp.Problem(cp.Minimize(e@b_op),[O@b_op==rhs,u+dt*K@b_op>=0])  
            prob.solve()
            if prob.status != cp.OPTIMAL:
                print(prob.status)
            
            b = b_op.value
        else:
            b =rkm.b
        #update
        u += dt*K@b
        n += 1
        t += dt
        
        uu[:,n] = u.copy()
        bb[:,n] = b.copy()
        tt[n] = t
        #print('updated')

        
    return (tt[0:n],uu[:,0:n],bb[:,0:n])
        
        
    



## Testproblem

Atmospheric pollution model from Hundsdorfer & Verwer



In [ ]:

# Atmospheric pollution model from Hundsdorfer & Verwer

def k1(t):
    th = t/3600  # Time in hours
    thbar = th - 24*(th//24)
    if thbar>4 and thbar<20:
        sec = (np.sin(np.pi/16 *(thbar-4)))**0.2
        return 1.e-5*np.exp(7.*sec)
    else:
        return 1.e-40

k3 = 1.e-16
k2 = 1.e5
sigma2 = 1.e6

def f_pollution(t,u):
    du = np.zeros(4)
    du[0] = k1(t)*u[2] - k2*u[0]
    du[1] = k1(t)*u[2] - k3*u[1]*u[3] + sigma2
    du[2] = k3*u[1]*u[3]-k1(t)*u[2]
    du[3] = k2*u[0] - k3*u[1]*u[3]
    return du

def Jacobian(t,u):
    J = np.array([
        [-k2  ,0       ,   k1(t),         0],
        [0    ,-k3*u[3],   k1(t),  -k3*u[1]],
        [0    , k3*u[3],  -k1(t),  -k3*u[1]],
        [k2   ,-k3*u[3],       0,  -k3*u[1]]
    ])
    return J

In [ ]:
#Solution of ODE blows up after a handfull steps. Further investigating the properties of the test Problem
t_final = 3600*24*6
dt = 0.1
u0 = np.array([0.,1.3e8,5.e11,8.e11])
J = Jacobian(0.,u0)
linalg.eig(J)

In [ ]:
# Reference implementation from Nodepy
from nodepy import ivp, rk

rk_ssp104 = rk.loadRKM('SSP104')


testp = ivp.IVP(f=f_pollution,u0=u0,T=1)

t,u =rk_ssp104(testp,dt=0.0001)

plt.plot(t,u)

In [ ]:
t_final = 3600 #3600*24*6
dt = 0.0001
u0 = np.array([0.,1.3e8,5.e11,8.e11])


t, u, b = RK_variable_b(ssp104,dt,f_pollution,w0=u0,t_final=t_final,b_fixed=True)
#plt.plot(np.array(t)/3600/24,u[0,:],'-k');
#plt.xlim(0,6);

In [ ]:
#u.shape
plt.plot(np.array(t)/3600,u[0,:],'-o');
plt.plot(np.array(t)/3600,u[1,:],'-o');
plt.plot(np.array(t)/3600,u[2,:],'-o');
plt.plot(np.array(t)/3600,u[3,:],'-o');

In [ ]:
for i in range(10):
    plt.plot(t,b[i,:])


## Testproblem Time integrator

In [ ]:
def f_sin(t,u):
    return -np.sin(t)

def f_cos(t,u):
    return np.cos(t)

def f_const(t,u):
    return 1

u0 =np.array([2.0])

t,u,b = RK_variable_b(ssp104,0.01,f_sin,w0=u0,t_final=10,b_fixed=False)


In [ ]:
plt.plot(t,u[0,:])

In [ ]:
for i in range(10):
    plt.plot(t,b[i,:])



## Testproblem Harmonic Oscilator

In [ ]:
def f_A(t,u):
    A = np.array([[0,-1],[1,0]])
    c = np.array([1,1])
    #print(u)
    return A@(u-c)

u0 =np.array([1.,0.])

t,u,b = RK_variable_b(ssp104,0.1,f_A,w0=u0,t_final=100,b_fixed=False)



In [ ]:
plt.plot(t,u[0,:])
plt.plot(t,u[1,:])

In [ ]:
for i in range(4):
    plt.plot(t,b[i,:])



## Testproblem from Kopecz and Meister 2018

In [ ]:
def f_lin_I(t,u):
    a = 5 # a>0
    A = np.array([[-a,1],[a,-1]])
    return A@u

u0 =np.array([0.9,0.1])

t,u,b = RK_variable_b(ssp104,2,f_lin_I,w0=u0,t_final=100,b_fixed=False)



In [ ]:
plt.plot(t,u[0,:])
plt.plot(t,u[1,:])

In [ ]:
plt.plot(t,b.T)

In [ ]:
ssp104.real_stability_interval()

In [ ]:
#Nonlinear test problem
def f_nonlin(t,u):
    a = 0.3
    du = np.zeros(3)
    du[0] = -(u[0]*u[1])/(u[0]+1)
    du[1] = (u[0]*u[1])/(u[0]+1) -a*u[1]
    du[2] = a*u[1]
    return du

u0 = np.array([9.98,0.01,0.01])

t,u,b = RK_variable_b(ssp104,3,f_nonlin,w0=u0,t_final=30,b_fixed=True)



In [ ]:
plt.plot(t,u[0,:])
plt.plot(t,u[1,:])
plt.plot(t,u[2,:])

plt.grid()

In [ ]:
# Brusselator test problem
def f_brusselator(t,u):
    k1 =1
    k2 =1
    k3 =1
    k4 =1
    k5 =1
    k6 =1
    du = np.zeros(6)
    du[0] = -k1*u[0]
    du[1] = -k2*u[1]*u[4]
    du[2] = k3*u[1]*u[4]
    du[3] = k4*u[4]
    du[4] = k1*u[0] - k2*u[1]*u[4] + k3*u[4]**2*u[5] - k4*u[4]
    du[5] = k2*u[1]*u[4] - k3*u[4]**2*u[5]
    return du

u0 = np.array([10.,10.,0.,0.,0.1,0.1])


t,u,b = RK_variable_b(ssp104,0.5,f_brusselator,w0=u0,t_final=6,b_fixed=True)

In [ ]:
plt.plot(t,u[0,:])
plt.plot(t,u[1,:])
plt.plot(t,u[2,:])
plt.plot(t,u[3,:])
plt.plot(t,u[4,:])
plt.plot(t,u[5,:])

In [ ]:
#Robertson test problem, stiff
def f_robertson(t,u):
    a = 0.3
    du = np.zeros(3)
    du[0] = 1e4 *u[1]*u[2] - 0.04*u[0]
    du[1] = 0.04 *u[0] - 1e4*u[1]*u[2] - 3e7*u[1]**2
    du[2] = 3e7*u[1]**2
    return du

u0 = np.array([1.,0.,0.])

t,u,b = RK_variable_b(ssp104,1e-6,f_robertson,w0=u0,t_final=1,b_fixed=True)

# Decay with Production

ODE of form $u'(t) = - \lambda u(t) + f(t)$ with $f(t) \geq 0 $ 

In [ ]:
def production(t):
    return 0.05*(1+np.sin(t))

def f_decay(t,u):
    l = 1
    return -l*u+production(t)

u0 = np.array([1.])

t,u,b = RK_variable_b(ssp104,10,f_decay,w0=u0,t_final=200,b_fixed=False)

In [ ]:
plt.plot(t,u[0,:])
plt.grid()

In [ ]:
plt.plot(t,b.T)

# Advection

In [ ]:
N=50
x = np.linspace(0,1,N)
dx = x[1]-x[0]

def f_centered_advection(t,u):
    du = np.zeros_like(u)
    du[1:-1] = (u[2:]-u[:-2])
    du[0] = u[1]-u[-1]
    du[-1] = u[0]-u[-2]
    return -du/(2*dx)

u0 = (x<0.5)*1.
u0 = (x<0.5)+0.3
#u0 = np.sin(2*np.pi*x)+1.

dt = 2.5*dx

t,u,b = RK_variable_b(ssp104,dt,f_centered_advection,w0=u0,t_final=1,b_fixed=False)

In [ ]:
plt.pcolor(u[:,::1])
plt.colorbar()

In [ ]:
plt.plot(u[:,-1])

In [ ]:
plt.plot(t,b.T)